# Лабораторна робота №2  
## Тема: Методи одномірної оптимізації
## Виконав: Варакута Олександр 
## група КІ-24-1

## Мета: навчитися розв’язувати задачу одновимірної нелінійної безумовної оптимізації.

### Умова задачі

Знайти мінімум функції
$$
F(x) = (5 - x^2)\cdot7x
$$

на інтервалі
$$
x \in [-2.5;\,-0.5]
$$

з точністю
$$
\varepsilon = 0.1.
$$


In [1]:
import numpy as np
import pandas as pd
import sympy as sp
from scipy.optimize import minimize_scalar


In [2]:
# Означення функції
def f(x):
    return (5 - x**2) * 7 * x


## 1. Метод Ньютона

Метод Ньютона використовує першу та другу похідні функції.
Ітераційна формула має вигляд:
$$
x_{k+1} = x_k - \frac{F'(x_k)}{F''(x_k)}.
$$


In [6]:
# Символьні похідні
x = sp.symbols('x')
F_sym = (5 - x**2) * 7*x
F1_sym = sp.diff(F_sym, x)
F2_sym = sp.diff(F1_sym, x)

# Числові функції
F = sp.lambdify(x, F_sym, 'numpy')
F1 = sp.lambdify(x, F1_sym, 'numpy')
F2 = sp.lambdify(x, F2_sym, 'numpy')

# Початкові дані
x0 = -1.5
eps = 0.1

rows = []

while True:
    x_next = x0 - F1(x0) / F2(x0)
    rows.append([x0, x_next, F(x_next)])
    if abs(x_next - x0) < eps:
        break
    x0 = x_next

df_newton = pd.DataFrame(
    rows,
    columns=["x_k", "x_{k+1}", "F(x_{k+1})"]
)

df_newton




,x_k,x_{k+1},F(x_{k+1})
0,-1.500000,-1.305556,-30.117434
1,-1.305556,-1.291076,-30.123204


In [7]:
x_min_newton = df_newton.iloc[-1, 1]
f_min_newton = F(x_min_newton)

x_min_newton, f_min_newton


(np.float64(-1.2910756501182032), np.float64(-30.12320362507165))

## 2. Метод Фібоначчі

Метод Фібоначчі належить до інтервальних методів оптимізації
і не потребує обчислення похідних.

Кількість ітерацій визначається з умови:
$$
F_n \ge \frac{b-a}{\varepsilon}.
$$


In [8]:
def fibonacci_numbers(n):
    fib = [1, 1]
    for _ in range(2, n):
        fib.append(fib[-1] + fib[-2])
    return fib

def fibonacci_search(f, a, b, eps):
    fib = fibonacci_numbers(30)
    rows = []

    for k in range(1, len(fib)-2):
        x1 = a + fib[-k-2]/fib[-k] * (b - a)
        x2 = a + fib[-k-1]/fib[-k] * (b - a)

        f1 = f(x1)
        f2 = f(x2)

        rows.append([k, a, b, x1, x2, f1, f2])

        if f1 > f2:
            a = x1
        else:
            b = x2

        if abs(b - a) < eps:
            break

    x_min = (a + b) / 2
    return x_min, f(x_min), pd.DataFrame(
        rows,
        columns=["k", "a", "b", "x1", "x2", "F(x1)", "F(x2)"]
    )

a, b = -2.5, -0.5
x_min_fib, f_min_fib, df_fib = fibonacci_search(f, a, b, eps)

df_fib


,k,a,b,x1,x2,F(x1),F(x2)
0,1,-2.500000,-0.500000,-1.736068,-1.263932,-24.135643,-30.103487
1,2,-1.736068,-0.500000,-1.263932,-0.972136,-30.103487,-27.593750
2,3,-1.736068,-0.972136,-1.444272,-1.263932,-29.461054,-30.103487
3,4,-1.444272,-0.972136,-1.263932,-1.152476,-30.103487,-29.621621
4,5,-1.444272,-1.152476,-1.332816,-1.263932,-30.075274,-30.103487
5,6,-1.332816,-1.152476,-1.263932,-1.221360,-30.103487,-29.994106
6,7,-1.332816,-1.221360,-1.290243,-1.263932,-30.123189,-30.103487


In [9]:
x_min_fib, f_min_fib


(-1.2983738762559494, -30.12172464193289)

## 3. Метод `minimize_scalar()` (SciPy)

Для перевірки результатів використаємо функцію `minimize_scalar()`
з пакета SciPy з методом `bounded`.


In [11]:
from scipy.optimize import minimize_scalar

# Означення функції
def f(x):
    return (5 - x**2) * 7 * x

# Мінімізація на відрізку [-2.5, -0.5]
res = minimize_scalar(
    f,
    bounds=(-2.5, -0.5),
    method="bounded"
)

# Вивід результатів
print("Оптимізація успішна:", res.success)
print("Повідомлення:", res.message)
print("Точка мінімуму x =", res.x)
print("Мінімальне значення F(x) =", res.fun)


Оптимізація успішна: True
Повідомлення: Solution found.
Точка мінімуму x = -1.290994461584702
Мінімальне значення F(x) = -30.12320380383546


## 4. Порівняння результатів


In [13]:
pd.DataFrame({
    "Метод": ["Ньютона", "Фібоначчі", "SciPy"],
    "x_min": [x_min_newton, x_min_fib, res.x],
    "F(x_min)": [f_min_newton, f_min_fib, res.fun]
})


,Метод,x_min,F(x_min)
0,Ньютона,-1.291076,-30.123204
1,Фібоначчі,-1.298374,-30.121725
2,SciPy,-1.290994,-30.123204


## Висновки

У ході лабораторної роботи було реалізовано три чисельні методи
пошуку мінімуму функції однієї змінної.

Метод Ньютона забезпечує швидку збіжність, але потребує обчислення похідних
і залежить від вибору початкового наближення.

Метод Фібоначчі є більш надійним, оскільки не використовує похідних,
але потребує більшої кількості ітерацій.

Функція `minimize_scalar()` пакета SciPy показала стабільний результат
і підтвердила коректність отриманих розв’язків.


## Контрольні питання

### 1. Чим відрізняються прямі і непрямі методи розв’язання задачі
одновимірної безумовної нелінійної оптимізації?

Прямі методи оптимізації — це методи, які не використовують похідні
функції і ґрунтуються лише на обчисленні значень функції в окремих точках
інтервалу.

До прямих методів належать:
- метод поділу навпіл;
- метод золотого перетину;
- метод Фібоначчі.

Основні характеристики прямих методів:
- не потребують обчислення похідних;
- можуть застосовуватися до недиференційовних функцій;
- мають гарантовану збіжність для унімодальних функцій;
- зазвичай потребують більшої кількості ітерацій.

---

Непрямі методи оптимізації — це методи, які використовують похідні
функції для визначення напрямку та швидкості збіжності до точки екстремуму.

До непрямих методів належать:
- метод Ньютона;
- метод градієнтного спуску (у багатовимірному випадку).

Основні характеристики непрямих методів:
- використовують першу та/або другу похідні;
- забезпечують швидку (часто квадратичну) збіжність;
- чутливі до вибору початкового наближення;
- можуть не збігатися при відсутності гладкості функції.

---

Порівняльна характеристика:

| Критерій | Прямі методи | Непрямі методи |
|--------|-------------|---------------|
| Використання похідних | Ні | Так |
| Швидкість збіжності | Повільна | Висока |
| Стійкість | Висока | Нижча |
| Вимоги до функції | Мінімальні | Диференційовність |

---

### 2. Сформулюйте необхідну та достатню умови існування екстремуму функції

Нехай функція $F(x)$ визначена та диференційовна в околі точки $x_0$.

---

Необхідна умова існування екстремуму:

Якщо функція $F(x)$ має локальний екстремум у точці $x_0$ і є диференційовною
в цій точці, то її перша похідна дорівнює нулю:
$$
F'(x_0) = 0.
$$

Точки, в яких виконується ця умова, називаються **стаціонарними**.

---

**Достатня умова існування екстремуму (через другу похідну):**

Нехай
$$
F'(x_0) = 0
$$
і друга похідна існує в точці $x_0$.

Тоді:
- якщо
$$
F''(x_0) > 0,
$$
то в точці $x_0$ функція має **локальний мінімум**;

- якщо
$$
F''(x_0) < 0,
$$
то в точці $x_0$ функція має **локальний максимум**;

- якщо
$$
F''(x_0) = 0,
$$
то за цією ознакою характер екстремуму визначити неможливо, і необхідно
досліджувати похідні вищих порядків або поведінку функції в околі точки.
